In [1]:
import pandas as pd

In [35]:
train_df = pd.read_excel('Data_Train.xlsx')
test_df = pd.read_excel('Data_Test.xlsx')
test_copy=test_df

## Analyzing data

In [36]:
train_df.head(10)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74
5,Hyundai EON LPG Era Plus Option,Hyderabad,2012,75000,LPG,Manual,First,21.1 km/kg,814 CC,55.2 bhp,5.0,2.35
6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,3.50
7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8.0,17.50
8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,Diesel,Manual,First,20.54 kmpl,1598 CC,103.6 bhp,5.0,5.20
9,Tata Indica Vista Quadrajet LS,Chennai,2012,65932,Diesel,Manual,Second,22.3 kmpl,1248 CC,74 bhp,5.0,1.95


In [37]:
train_df.dtypes

Name                  object
Location              object
Year                   int64
Kilometers_Driven      int64
Fuel_Type             object
Transmission          object
Owner_Type            object
Mileage               object
Engine                object
Power                 object
Seats                float64
Price                float64
dtype: object

In [38]:
train_df.isna().sum()

Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               2
Engine               36
Power                36
Seats                42
Price                 0
dtype: int64

In [39]:
train_df.describe()

,Year,Kilometers_Driven,Seats,Price
count,6019.000000,6.019000e+03,5977.000000,6019.000000
mean,2013.358199,5.873838e+04,5.278735,9.479468
std,3.269742,9.126884e+04,0.808840,11.187917
min,1998.000000,1.710000e+02,0.000000,0.440000
25%,2011.000000,3.400000e+04,5.000000,3.500000
50%,2014.000000,5.300000e+04,5.000000,5.640000
75%,2016.000000,7.300000e+04,5.000000,9.950000
max,2019.000000,6.500000e+06,10.000000,160.000000


## Approach

In [40]:
# Splitting Training data(train_df) into train and test dfs to calculate accuracy. Will then use model on test_df

## Cleaning data 

In [41]:
def transform_owner(x):
    if x=='First':
        return 1
    elif x=='Second':
        return 2
    elif x=='Third':
        return 3
    return 4

In [42]:
def transform_transmission(x):
    if x=='Manual':
        return 1
    return 4

In [43]:
def cleanDF(df):
    # Remove extra characters like units and convert blank values to NAs.
    df['Engine']=df['Engine'].astype(str).str.replace('\D+', '')
    df['Engine']=pd.to_numeric(df['Engine'], errors='coerce')
    df['Power']=df['Power'].astype(str).str.replace('\D+', '')
    df['Power']=pd.to_numeric(df['Power'], errors='coerce')
    df['Mileage']=df['Mileage'].astype(str).str.replace('\D+', '')
    df['Mileage']=pd.to_numeric(df['Mileage'], errors='coerce',downcast='float')
    
    # If fuel type is CNG/LPG then units is km/kg. 1 kg roughly equals 1.5 litres. So dividing by 1.5 to convert km/kg to kml
    mask = (df['Fuel_Type'] == "CNG") | (df['Fuel_Type'] == "LPG")
    df['Mileage'][mask] = df['Mileage']/1.5
    
    #Remove NAS
    df = df.dropna()
    
    #Convert Categorical column-> Numerical Data
    df['Owner_Type']= df['Owner_Type'].apply(transform_owner)
    df['Transmission']= df['Transmission'].apply(transform_transmission)
    
    # Apply one hot encoding for Fuel Type
    a = pd.get_dummies(df['Fuel_Type'])
    df[['CNG','Diesel','LPG','Petrol']] = a
    df = df.drop(columns=['Fuel_Type'])
    
    return df

In [44]:
train_df = cleanDF(train_df)

C:\Users\Peehu\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Peehu\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Peehu\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [45]:
train_df.head()

,Name,Location,Year,Kilometers_Driven,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,CNG,Diesel,LPG,Petrol
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,1,1,177.333328,998.0,5816.0,5.0,1.75,1,0,0,0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,1,1,1967.000000,1582.0,1262.0,5.0,12.50,0,1,0,0
2,Honda Jazz V,Chennai,2011,46000,1,1,182.000000,1199.0,887.0,5.0,4.50,0,0,0,1
3,Maruti Ertiga VDI,Chennai,2012,87000,1,1,2077.000000,1248.0,8876.0,7.0,6.00,0,1,0,0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,4,2,152.000000,1968.0,1408.0,5.0,17.74,0,1,0,0


In [46]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [47]:
# Splitting cleaned data into X and Y
X = train_df[['Year','Kilometers_Driven','Transmission','Owner_Type','Mileage','Engine',
              'Power','Seats','CNG','Diesel','LPG','Petrol']]
Y= train_df['Price']

In [48]:
X.head()

,Year,Kilometers_Driven,Transmission,Owner_Type,Mileage,Engine,Power,Seats,CNG,Diesel,LPG,Petrol
0,2010,72000,1,1,177.333328,998.0,5816.0,5.0,1,0,0,0
1,2015,41000,1,1,1967.000000,1582.0,1262.0,5.0,0,1,0,0
2,2011,46000,1,1,182.000000,1199.0,887.0,5.0,0,0,0,1
3,2012,87000,1,1,2077.000000,1248.0,8876.0,7.0,0,1,0,0
4,2013,40670,4,2,152.000000,1968.0,1408.0,5.0,0,1,0,0


In [49]:
# Noramlising data
from sklearn import preprocessing
x = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled)

## Splitting train_df further into train and test so that we can calculate accuracy on test data

In [50]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

### Linear Regression

In [51]:
regressor = LinearRegression()  
regressor.fit(x_train, y_train) #training the algorithm

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [52]:
Y_trainPredL = regressor.predict(x_test)

## Analyzing Results

In [53]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, Y_trainPredL))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, Y_trainPredL))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, Y_trainPredL)))

Mean Absolute Error: 4.054343715314593
Mean Squared Error: 58.396333980550075
Root Mean Squared Error: 7.64174940576764


### K Means

In [54]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(x_train, y_train)
Y_trainPredK = neigh.predict(x_test)

## Analyzing Results

In [55]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, Y_trainPredK))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, Y_trainPredK))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, Y_trainPredK)))

Mean Absolute Error: 2.3325063829787234
Mean Squared Error: 40.0661155106383
Root Mean Squared Error: 6.32978005231132


### Decision Tree Regressor

In [56]:
from sklearn.tree import DecisionTreeRegressor
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_1.fit(x_train, y_train)
Y_trainPredT = regr_1.predict(x_test)

In [57]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, Y_trainPredT))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, Y_trainPredT))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, Y_trainPredT)))

Mean Absolute Error: 4.235123373760477
Mean Squared Error: 81.4598342503668
Root Mean Squared Error: 9.025510193355654


## Predict on unknown test_df

In [58]:
test_df= cleanDF(test_df)
X_test = test_df[['Year','Kilometers_Driven','Transmission','Owner_Type','Mileage','Engine',
              'Power','Seats','CNG','Diesel','LPG','Petrol']]
xt = X_test.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X_test = pd.DataFrame(x_scaled)

C:\Users\Peehu\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Peehu\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Peehu\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

## Value predictions for test_df

In [59]:
y_pred_linear = regressor.predict(X_test) # using linear regression
y_pred_kmeans = neigh.predict(X_test) # using kmeans
y_pred_dtree = regr_1.predict(X_test) # using kmeans

In [60]:
y_pred_linear

array([-1.52917074, 10.34393069,  0.25106472, ...,  9.2776186 ,
        0.17930331, -1.08791244])

In [61]:
y_pred_kmeans

array([ 1.875, 10.575,  4.45 , ...,  4.2  ,  2.775,  2.45 ])

In [62]:
y_pred_dtree

array([3.78995729, 6.81219168, 3.78995729, ..., 9.97012658, 3.78995729,
       3.78995729])

In [63]:
import seaborn as sns 
import matplotlib.pyplot as plt 

In [70]:
'''
test_copy['Predict_Regression']=y_pred_linear
test_copy['Predict_Kmeans']=y_pred_Kmeans
test_copy['Predict_Decision']=y_pred_dtree
test_copy.head(10)
'''
print(len(y_pred_linear))
print(len(y_pred_kmeans))
print(len(y_pred_dtree))
print(len(test_copy))


5872
5872
5872
1234


In [75]:
X_test.head()


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.571429,0.011051,0.0,0.000000,0.063130,0.069594,0.053301,0.375,1.0,0.0,0.0,0.0
1,0.809524,0.006282,0.0,0.000000,0.700249,0.178266,0.011313,0.375,0.0,1.0,0.0,0.0
2,0.619048,0.007051,0.0,0.000000,0.064792,0.106997,0.007855,0.375,0.0,0.0,0.0,1.0
3,0.666667,0.013359,0.0,0.000000,0.739409,0.116115,0.081514,0.625,0.0,1.0,0.0,0.0
4,0.714286,0.006231,1.0,0.333333,0.054112,0.250093,0.012659,0.375,0.0,1.0,0.0,0.0


In [80]:
r=y_pred_kmeans[::11]
len(r)

534